In [ ]:
!pip install mplfinance

# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf

# This library support to get data from Trading view

!pip install --upgrade --no-cache-dir git+https://github.com/rongardF/tvdatafeed.git

# Use installed library to get data from Trading View

from tvDatafeed import TvDatafeed, Interval

username = 'shenaal1992@gmail.com'
password = 'Menushiiloveu22'

tv = TvDatafeed(username, password)

tv = TvDatafeed()

  Cloning https://github.com/rongardF/tvdatafeed.git to /tmp/pip-req-build-dal00pnq
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed.git /tmp/pip-req-build-dal00pnq
  Resolved https://github.com/rongardF/tvdatafeed.git to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression

def build_multi_input_multi_output_model(x1_train, y1_train):
    model = MultiOutputRegressor(LinearRegression())
    model.fit(x1_train, y1_train)  # x_train and y_train are multi-dimensional arrays
    return model

In [ ]:
def get_training(date_time_of_buy_or_sell,symbol='AAPL',exchange='NASDAQ'):
  df  = tv.get_hist(symbol=symbol,exchange=exchange,interval=Interval.in_4_hour,n_bars=10000)
  index_position = df.index.get_loc(date_time_of_buy_or_sell)
  previous_data = df.iloc[max(0, index_position - 3999):index_position + 1]

  close = previous_data.filter(['close'])
  close_price = close.values

  # Initialize Scaler
  from sklearn.preprocessing import MinMaxScaler
  scaler_close = MinMaxScaler()

  # Scaling the data
  scaled_data_close = scaler_close.fit_transform(close_price)

  sc_data_close  = []
  for i in scaled_data_close:
    sc_data_close.append(i.tolist()[0])

  x_train = []
  y_train = []

  input_sequence_length= 7
  output_sequence_length = 2


  for i in range(input_sequence_length, len(sc_data_close)-1):
    x_train.append(sc_data_close[i-input_sequence_length:i])
    y_train.append(sc_data_close[i:i+output_sequence_length])

  model_ = build_multi_input_multi_output_model(x_train, y_train)
  predicted  = model_.predict(np.array(sc_data_close[-7:]).reshape(1, -1))
  inversed_prediction = scaler_close.inverse_transform(predicted)

  inversed_prediction_1 = inversed_prediction[0][0]
  inversed_prediction_2 = inversed_prediction[0][1]



  return inversed_prediction_1,inversed_prediction_2


In [ ]:
def get_return(symbol='AAPL',exchange='NASDAQ'):
  df  = tv.get_hist(symbol=symbol,exchange=exchange,interval=Interval.in_4_hour,n_bars=1000)

  # Define the window size for moving average (e.g., 20 periods)
  window = 20

  # Calculate the moving average
  df['MA'] = df['close'].rolling(window=window).mean()

  # Calculate the standard deviation
  df['STD'] = df['close'].rolling(window=window).std()

  # Calculate Bollinger Bands for 2 and 3 standard deviations
  df['Upper_Band_2STD'] = df['MA'] + (df['STD'] * 2)
  df['Lower_Band_2STD'] = df['MA'] - (df['STD'] * 2)
  df['Upper_Band_3STD'] = df['MA'] + (df['STD'] * 3)
  df['Lower_Band_3STD'] = df['MA'] - (df['STD'] * 3)

  # Initialize variables for the trading strategy
  initial_money = 1000
  money = initial_money
  shares = 0
  buying_points =[]
  selling_points =[]

  # Loop through the DataFrame to apply the trading strategy
  for i in range(len(df)):
      if df['close'].iloc[i] < df['Lower_Band_3STD'].iloc[i] and money > 0:
          date_time_of_buy = df.index[i]
          prediction_1,prediction_2 = get_training(date_time_of_buy)

          if prediction_2 < df['close'].iloc[i]:
              # Buy stock
              shares = money / df['close'].iloc[i+2]
              money = 0  # All money is invested in shares
              buying_points.append(df.index[i+2])  # Store the index of the buying point
              print(f"Bought on {df.index[i+2].date()} at {df['close'].iloc[i+2]} based on prediction")
          else:
              # Buy stock at current price
              shares = money / df['close'].iloc[i]
              money = 0  # All money is invested in shares
              buying_points.append(df.index[i])  # Store the index of the buying point
              print(f"Bought on {df.index[i].date()} at {df['close'].iloc[i]}")

      elif df['close'].iloc[i] > df['Upper_Band_2STD'].iloc[i] and shares > 0:
          date_time_of_sell = df.index[i]
          prediction_1,prediction_2 = get_training(date_time_of_sell)

          if prediction_2 > df['close'].iloc[i]:
              # Sell stock
              money = shares * df['close'].iloc[i+2]
              shares = 0  # All shares are sold
              selling_points.append(df.index[i+2])  # Store the index of the selling point
              print(f"Sold on {df.index[i+2].date()} at {df['close'].iloc[i+2]} based on prediction")
          else:
              # Sell stock at current price
              money = shares * df['close'].iloc[i]
              shares = 0  # All shares are sold
              selling_points.append(df.index[i])
              print(f"Sold on {df.index[i].date()} at {df['close'].iloc[i]}")

  # Calculate the final value
  final_value = money + (shares * df['close'].iloc[-1])
  total_return = final_value - initial_money

  print(f"Initial money: {initial_money}")
  print(f"Final value: {final_value}")
  print(f"Total return: {total_return}")

  return total_return

In [ ]:
tickers = [
    "AAPL",
    "MSFT",
    "AMZN",
    "NVDA",
    "GOOG",
    "GOOGL",
    "TSLA",
    "META",
    "AVGO",
    "PEP",
    "COST",
    "CSCO",
    "AMD",
    "CMCSA",
    "TXN",
    "TMUS",
    "QCOM",
    "AMGN",
    "HON",
    "BKNG"
]
re = []
for i in tickers:
  ret = get_return(symbol= i,exchange='NASDAQ')
  re.append(ret)

data = { 'tickers' : ["AAPL", "MSFT",
    "AMZN",
    "NVDA",
    "GOOG",
    "GOOGL",
    "TSLA",
    "META",
    "AVGO",
    "PEP",
    "COST",
    "CSCO",
    "AMD",
    "CMCSA",
    "TXN",
    "TMUS",
    "QCOM",
    "AMGN",
    "HON",
    "BKNG"
],
         'return' : re }
  # Create the DataFrame
df = pd.DataFrame(data)
df

Bought on 2022-11-29 at 140.54
Sold on 2023-01-23 at 142.27
Bought on 2023-08-04 at 184.98
Sold on 2023-08-24 at 177.74 based on prediction
Bought on 2024-01-02 at 185.29
Sold on 2024-01-19 at 191.56 based on prediction
Bought on 2024-03-05 at 170.47 based on prediction
Sold on 2024-03-19 at 175.64 based on prediction
Initial money: 1000
Final value: 1036.1010107351476
Total return: 36.10101073514761
Initial money: 1000
Final value: 1000.0
Total return: 0.0
Initial money: 1000
Final value: 1000.0
Total return: 0.0
Bought on 2024-04-22 at 794.5 based on prediction
Sold on 2024-05-23 at 1059.64
Initial money: 1000
Final value: 1333.71932032725
Total return: 333.71932032725
Bought on 2023-10-25 at 127.41
Sold on 2023-11-17 at 136.13 based on prediction
Initial money: 1000
Final value: 1068.4404677811788
Total return: 68.44046778117877
Bought on 2023-10-25 at 126.385
Sold on 2023-11-17 at 134.33 based on prediction
Initial money: 1000
Final value: 1062.8634727222377
Total return: 62.863472

,tickers,return
0,AAPL,36.101011
1,MSFT,0.000000
2,AMZN,0.000000
3,NVDA,333.719320
4,GOOG,68.440468
5,GOOGL,62.863473
6,TSLA,250.849692
7,META,527.525283
8,AVGO,0.000000
9,PEP,128.451679


In [ ]:
df

,tickers,return
0,AAPL,36.101011
1,MSFT,0.000000
2,AMZN,0.000000
3,NVDA,333.719320
4,GOOG,68.440468
5,GOOGL,62.863473
6,TSLA,250.849692
7,META,527.525283
8,AVGO,0.000000
9,PEP,128.451679


In [ ]:
df1  = tv.get_hist(symbol='PEP',exchange='NASDAQ',interval=Interval.in_4_hour,n_bars=1000)
df1.head(1)

,symbol,open,high,low,close,volume
datetime,,,,,,
2022-05-26 17:30:00,NASDAQ:PEP,169.89,171.08,169.88,170.19,77279.0


In [ ]:
df1.tail(1)

,symbol,open,high,low,close,volume
datetime,,,,,,
2024-05-24 17:30:00,NASDAQ:PEP,178.59,178.81,177.88,177.99,98949.0


In [ ]:
get_return(symbol='MSFT',exchange='NASDAQ')

Initial money: 1000
Final value: 1000.0
Total return: 0.0


In [ ]:
get_return(symbol='TSLA',exchange='NASDAQ')

Bought on 2023-04-20 at 164.35
Sold on 2023-05-08 at 172.21 based on prediction
Bought on 2023-10-20 at 215.36 based on prediction
Sold on 2023-11-14 at 236.091
Bought on 2024-01-25 at 181.93
Sold on 2024-02-15 at 198.11
Initial money: 1000
Final value: 1250.8496919566621
Total return: 250.84969195666213


In [ ]:
get_return(symbol='AMD',exchange='NASDAQ')

Bought on 2024-04-04 at 165.82
Sold on 2024-05-16 at 162.62 based on prediction
Initial money: 1000
Final value: 980.7019659872151
Total return: -19.29803401278491
